
# Activities (Optional)

In this practicum you will learn how to setup an MVC application with a template engine. The template engine that we will use is called Razor. Razor is a __markup syntax__ that allows to embed server-based code into web pages by means of C#.



Before showing how to setup such a project we will first take a look at what is HTML. This is needed to understand the fundamental components that are necessary to build the View entity in our MVC.

#### Introduction to HTML
HTML (Hyper Text Markup Language) is a markup language (so it is not a programming langauge) for creating _static_ web pages. Markups are a set of tags that describe how text should be displayed on a web page.

Every HTML page is made by the following items:
    - Head, contains info about the webpage
    - Body, contains the actual body to display on the screen
    
<img src="1.JPG">

When we run the code above the output on your browser will be:

<img src="2.JPG">


Since pages in HTML are static, typical programming elements such as while loops, calling a function, etc. are not found. This is the reason why to allow dynamism in pages we need expressive tools, such as a programming language. The choice of where to simulate such dynamic behavior is currently not trivial, since we are simulating the behavior of distributed applications. The possible options to run code in a html are different and depend on the context.
Code could be executed completely on the browser (client side), completely on the server, or a trade-off. Today we will see the second option, that is how to run code on the server. 


In order to achieve this, we will use Razor together with ASP.net. In Razor a template file is made of a mix of literal text (a sort of HTML) and C# statements (not all C# statements are allowed [1]). When all required data for a view are available, the template engine will take care of embedding this data into the template and send the resulting page to the client. The page comes in the shape of a HTML page that can be interpreted by a common browser. __In a Razor template file, C# statements starts with the __@__ sign. This sign will be used to automatically start parsing a statement by the engine, otherwise the HTML code will be directly pushed without any further processing. __


<img src="0.jpg">

#### Setting up an MVC application with Razor


In the following we see a few examples of how to use Razor in a basic web application. In the end we will see how to present our Movie model with Razor in an MVC application.


##### Simple "datetime" view in Razor

To create a basic Razor project call the command `dotnet new razor -o Practicum4Razor` (the option -o will give the project the name specified, in this case 'Practicum4Razor') and then `dotnet restore`.

To test the code of this sample, remove first the content of the _Index.cshtml_ file present in the _Pages_ folder and add to it the following code:


```
@page
@using Practicum4Razor
@model IndexModel

<h1>Hello World</h1>
<h3>
    @Model.Message
</h3>
```

In the code above, the statement _@page_ converts the file into an MVC action, which means that it handles requests directly, without going through a controller. 
In addition when the index page is loaded a __Hello World__ message is presented with an additional message that comes from the model (_@Model.Message_). The message comes from the model which is defined in the file called _Index.cshtml.cs_. In this file we will define the shape of our model and how to access it.

Now we will add a value to the message to print on the screen together with the _Hello World_ message. More precisely, we will print the current time. Open the file _Index.cshtml.cs_ and copy the following code.


```
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.AspNetCore.Mvc;
using Microsoft.AspNetCore.Mvc.RazorPages;

namespace Practicum4Razor.Pages
{
    public class IndexModel : PageModel
    {

        public string Message { get; private set; }

        public void OnGet()
        {

            Message = DateTime.Now.ToString(); 
        }
    }
}

```

We can attach more data to present to the view by means of the _ViewData_ object, so we do not use the message property as in the previous example. _ViewData_ is a dictionary that can carry additional data that are not directly binded in the models' properties. In the following example in the model we assign as information to _ViewData_ the current time and a _Hello World_ message (surrounded by a html tag) respectively. 

```
namespace Practicum4Razor.Pages
{
    public class IndexModel : PageModel
    {

        public string Message { get; private set; } = "Razor Is Awesome";

        public void OnGet()
        {

            ViewData["Time"] = DateTime.Now;
            ViewData["Foo"] = "<h1>Hello World</h1>";
        }
    }
}

```

To visualize the data when the index page is opened use the following code. In the following code we print the message contained in the message property of our model, the time, and the hello world message. Note that to interpret the hello world message as HTML we use the _@Html.Raw_, which will take care of interpreting the string content as HTML.

```
@page
@using Practicum4Razor
@model IndexModel

<h2>Separate page model</h2>
<p>
    @Model.Message
    <div> 
        @ViewData["Time"]
        @Html.Raw(ViewData["Foo"])
    </div>  
</p> 

```

Remember to test always your code with the usual `dotnet run`.


##### Conditional logic and loops in Razor

Since we are in practice using C# we can manipulate more articulated data structures in Razor. For instance, in the following example we add a new attribute to our model that contains a collection of data time. The collection is populated with new values everytime we call the models' page.


```
namespace Practicum4Razor.Pages
{
    public class IndexModel : PageModel
    {
        public string Message { get; private set; } = "Razor Is Awesome";
        public List<DateTime> Times { get; private set; } = new List<DateTime>();
        public void OnGet()
        {
            for (int i = 0; i < 10; i++)
            {
                Times.Add(DateTime.Now);
            }
            ViewData["Foo"] = "<h1>Hello World</h1>";
        }
    }
}

```

To iterate and present all the times in the collection you can use a normal _foreach_ loop (remember always to use the _@_ that forces the engine to switch from the html to C# parser).

```
@page
@using Practicum4Razor
@model IndexModel

<h2>Separate page model</h2>
<p>
    @Model.Message
    <div> 
        @Html.Raw(ViewData["Foo"])
    </div>

    @foreach(var t in Model.Times){
        <ul>
            <li>@t</li>
        </ul>
    }  
</p> 

```

We can let our page interact with the model by means of forms. In the following example a form is made with an input box that manipulates the Name property of our model. The effect of the manipulation become persistent the moment we press the button submit, which takes care of selecting the correct post method.

```
@page
@using Practicum4Razor
@model IndexModel

<h2>Separate page model</h2>
<p>
    @Model.Message
    <div> 
        @Html.Raw(ViewData["Foo"])
    </div>

    @foreach(var t in Model.Times){
        <ul>
            <li>@t</li>
        </ul>
    }  

    <form method="POST">
        <div>Name: <input asp-for=@Model.Name /></div>
        <input type="submit" asp-page-handler="Test" value="Send" />
    </form>
</p> 
```

In the following the commented model is provided.

```
public class IndexModel : PageModel
{

    public string Message { get; private set; } = "Razor Is Awesome";
    public List<DateTime> Times { get; private set; } = new List<DateTime>();
    public void OnGet()
    {
        for (int i = 0; i < 10; i++)
        {
            Times.Add(DateTime.Now);
        }
        ViewData["Foo"] = "<h1>Hello World</h1>";
    }

    //We define Name as a binding property for this model, which means that it can 
    //contain information that for example come from a post request 
    [BindProperty]
    public string Name { get; set; }
    
    //Async method Test that serves the verb Post
    public async Task<IActionResult> OnPostTestAsync()
    {
        if (!ModelState.IsValid)
        {
            //In case the model contains invalid data we return the template associated to this
            //file (in our case the index) combined with the data of this model, but without
            //calling the get method above
            return Page();
        }

        Console.WriteLine("Received: " + Name);

        //Returns the template found in index combined with the data of this model.
        //Returning this page will automatically invoke the get method above
        return RedirectToPage("/Index");
    }
}
```


##### MVC and REST-style pages for the movie model (unrevised)

In this last section we will create an actual MVC project that will show the movies/actors made previously in Razor. More specifically we will be able to see all the movies/actors present in the database, create/delete movies/actors, and select movies that satisfy some predicates.


- Create an MVC project and build it by running the commands: `dotnet new mvc` and `dotnet restore`

- Follow the instructions of the previous practicum to add the movie model to this project. Do not forget to add the dependency to the database in the Startup file, and to run a migration/update once the db is correctly moved and setup.

- Now we need to to create a controller for every model of our database. You can do this by hand or automatically by calling the scaffolding command of aspnet. 
    - Add the package: `dotnet add package Microsoft.VisualStudio.Web.CodeGeneration.Design` and call also `dotnet restore`
    
    - To scaffold you need to call the following command: `dotnet aspnet-codegenerator controller -name MovieController -m Movie -dc MovieContext --relativeFolderPath Controllers --useDefaultLayout --referenceScriptLibraries` 
    
    - Make sure that the arguments of the command match the actual names of your model. In the case above we are creating a controller and a view (in razor) for the model _Movie_ found in _MovieContext_
    
    - Important, try to understand the scaffolding command above; for a detailed list of all the options run on your terminal the following command `dotnet aspnet-codegenerator controller --help`
    
    - Run the command above for every entity of your model
    
In the the projects' structure should look like the following

<img src="3.JPG">

- Run the project `dotnet run` and test the just made solution by navigating to _http://localhost:5000/movie_ or _http://localhost:5000/actor_

- As shown in the previous practicum you can add parameters to the methods of a controller to obtain customized behavior. In the following we alter the default _get_ our movies to select those which titles contain a given string (if no string is provided then all the movies are returned). To test the code below try to visit the page _http://localhost:5000/movie?searchString=lord_; you should be able to see only those movies which titles contain the word _lord_.

```
...
// GET: Movies
public async Task<IActionResult> Index(string searchString)
{
    var movies = _context.Movie.Select(m => m);

    if(!String.IsNullOrEmpty(searchString)){
        movies = _context.Movie.Where(m => m.Title.Contains(searchString));
    }

    return View(await movies.ToListAsync());
}

```

- At model level you can also add constraints to properties. These constraints, which come in the shape of tags, will tell Razor how to interpret the users behavior when interacting withe model through the web page. In the following we add a series of minimum requirements to the properties of our movies that must always be true.


```
using System.ComponentModel.DataAnnotations;
...
public class Movie {
    public int Id { get; set; }

    [Required(ErrorMessage = "Title is required")]
    [StringLength(25)]
    public string Title { get; set; }

    [Required(ErrorMessage = "Date is required")]
    [DisplayFormat(DataFormatString = "{0:yyyy-MM-dd}", ApplyFormatInEditMode = true)]
    public DateTime Release { get; set; }

    public List<Actor> Actors { get; set; }
    }
```

- Since we have changed the behavior of the properties of our model the template engine needs be notified, so to update the pages involved. More specifically we need to remake the pages that create and edit a movie. Thus, after adding the code above run the following scaffolding commands.

__dotnet aspnet-codegenerator view Movie Create -m Movie -dc MovieContext --relativeFolderPath Controllers --useDefaultLayout --referenceScriptLibraries -f__

__dotnet aspnet-codegenerator view Movie Edit -m Movie -dc MovieContext --relativeFolderPath Controllers --useDefaultLayout --referenceScriptLibraries -f__

- Important, try to understand the commands above and make sure that the arguments of the commands above match the actual names in your model.

- Note, if you call the scaffolding command introduced at the beginning of this chapter you will loose all the changes you made to to the controller, since the command in question not only overrides all the views, but also all the controllers.

- For the complete list of possible commands to scaffold the view of your application run the following command on the terminal __dotnet aspnet-codegenerator view --help__
    


    
References
    -[1] https://docs.microsoft.com/en-us/aspnet/core/mvc/views/razor